### Imports, Definitions

In [1]:
import math
import glob
import cv2
import numpy as np
import os
from keras.models import load_model
import time
import sys
from PIL import Image
import skimage.color as ski
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
import pywt
from skimage import exposure
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Crop and rotate minAreaRect
def crop_minAreaRect(img, rect):
    # rotate img
    angle = rect[2]
    rows,cols = img.shape[0], img.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv2.warpAffine(img,M,(cols,rows))

    # rotate bounding box
    rect0 = (rect[0], rect[1], 0.0)
    box = cv2.boxPoints(rect)
    pts = np.int0(cv2.transform(np.array([box]), M))[0]    
    pts[pts < 0] = 0

    # crop
    img_crop = img_rot[pts[1][1]:pts[0][1], 
                       pts[1][0]:pts[2][0]]

    return img_crop

# Crop and rotate box aligned with determinated angle
def box_w_angle(img, ref, center, radius, angle):
    
    rect = (center, (4*radius/3, 2*radius), angle)
    box = cv2.boxPoints(rect)
    box_angled = np.int0(box)
    
    # rotate img
    rows,cols = img.shape[0], img.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv2.warpAffine(img,M,(cols,rows))

    # rotate bounding box
    rect0 = (rect[0], rect[1], 0.0)
    box = cv2.boxPoints(rect)
    pts = np.int0(cv2.transform(np.array([box]), M))[0]    
    pts[pts < 0] = 0

    # crop
    img_crop = img_rot[pts[1][1]:pts[0][1], 
                       pts[1][0]:pts[2][0]]

    return box_angled, img_crop

# Char library
chlib = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'F', 'G', '', ''] # '' = T, ruido

Using TensorFlow backend.


In [2]:
# filtragem homomorfica
def fhomom(im, gl, gh, c, d):
       
    I_log = np.log1p(np.array(im,dtype="float"))    
    image = np.zeros((2*I_log.shape[0],2*I_log.shape[1]), dtype="float")

    for u in range(I_log.shape[0]):
        for v in range(I_log.shape[1]):
            image[u,v] = I_log[u,v]
            
    #CalcularaFFTdaimagem
    image = np.fft.fft2(image)
    image = np.fft.fftshift(image)

    H = get_homo_filt(image.shape[0],image.shape[1],gl,gh,c,d)
    image = image*H

    image = np.fft.fftshift(image)     
    image = np.abs(np.fft.ifft2(image))

    I_filt = np.zeros((I_log.shape), dtype="float")
    for u in range(I_filt.shape[0]):
        for v in range(I_filt.shape[1]):
            I_filt[u,v] = image[u,v]

    return np.uint8(np.exp(I_filt) - 1)


def get_homo_filt(P,Q,yl,yh,c,D0):
    _D = lambda u,v,P,Q : np.sqrt((u-P/2)**2+(v-Q/2)**2) 
    get_value = lambda yl,yh,c,D0,u,v : (yh-yl)*(1-np.exp(-c*((_D(u,v,P,Q)**2)/D0**2)))+yl
    
    filt = [[get_value(yl,yh,c,D0,u,v)
            for v in range(Q)] for u in range(P)]          
    return np.array(filt)


def logAbout_inverse(im,c):
    
    laplacian = np.c_[
    [-1,-1,-1],
    [-1,9,-1],
    [-1,-1,-1]
    ]
    
    gaussian = (1/16)*np.c_[
    [1,2,1],
    [2,4,2],
    [1,2,1]
    ]
    
    I = np.array(im,dtype="float")   
    I = 255 - I; #obtem o negativo da imagem
    I = convolve2d(I,gaussian, mode="same", boundary="symm")
    I_filter = convolve2d(I,laplacian, mode="same", boundary="symm")
    I_filter = I_filter - np.amin(I_filter)
    I_filter = np.log1p(I_filter,dtype="float")
    I_filt = (I_filter - np.amin(I_filter))*c
    I_filt = 255 - I_filt ##obtem o negativo do resultado
    return np.uint8(I_filt)


def improved_light(img,clip,c):
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=clip,tileGridSize=(8,8))
    im = clahe.apply(img)
    im = 255 - im
    I = np.array(im,dtype="float") 
    I = np.log1p(I)*c
    I = 255 - I
   
    gaussian = (1/16)*np.c_[
    [1,2,1],
    [2,4,2],
    [1,2,1]
    ]
    
    I = convolve2d(I,gaussian, mode="same", boundary="symm")
    I_blur = convolve2d(I,gaussian, mode="same", boundary="symm")
    I_mask = I - I_blur
    I_filter = I + I_mask
    
    return np.uint8(I_filter)
    
    
def wavelet_image(im,a,b,c):
    
    I = np.array(im,dtype=np.float64)  
    coeffs = pywt.dwt2(I, 'db10')
    cA, (cH, cV, cD) = coeffs
    
    # Equalization
    cA = exposure.equalize_hist(cA)
    cA = 5*cA*255*np.amax(cA)/1
    
    cH = cH*a
    cV = cV*b
    cD = cD*c

    reconstructed = pywt.idwt2((cA, (cH, cV, cD)), 'db10')
    return np.uint8(reconstructed)

In [3]:
# Load Convolutional Network Model (architecture + weights + optimizer state)
convNet = load_model("D:\\UFES\\RevisionWorkstation\\ConvNets\\modelCifarClahe")

### Get filenames

In [4]:
# SAVE START TIME
start_time = time.time()

# GET FILES FROM CHOSEN DIRECTORY
filenames = [fname for fname in glob.glob('D:\\UFES\\RevisionWorkstation\\Benchmark\\*.png')]
filenames.sort() 
total = len(filenames)

## Main Loop - TopHat + HMA + HMB

### Main Processing

In [24]:
# SAVE PROCESSING START TIME
mid_time = time.time()
progress = 0

# Auxiliary Counters
count = 0
case0 = 0
case1 = 0
case2 = 0
case3 = 0
zero = 0

errorImage = []
errorFile = []

# ------------------------------------------------------- #  
# MAIN LOOP
# ------------------------------------------------------- #  

for fileloc in filenames:
        
    # READ IMAGE
   
    try:
        imgx = cv2.imread(fileloc)
        imgf = imgx.copy()
    except:
        print('Read/Copy Img - ' + fileloc)
        continue

    # ------------------------------------------------------- #  
    # ROI EXTRACTION
    # ------------------------------------------------------- #  
    
    # Split Channels
    b,g,r = cv2.split(imgf)
    red = r - g 
    blue = b - g

    # Dilate/Threshold to get 1 contour only
    kernel_dil = np.ones((3,3),np.uint8)
    red = cv2.dilate(red, kernel_dil, iterations = 3)
    rets, thRed = cv2.threshold(red, 100, 255, cv2.THRESH_BINARY)

    # Find red contour
    contRed, hierqr = cv2.findContours(thRed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Order contours by size
    if(len(contRed) > 1):
        contRed = sorted(contRed, key=lambda x: cv2.contourArea(x))

    # Find extreme points of Red Area
    try:
        c = max(contRed, key=cv2.contourArea)
        extBotRed = tuple(c[c[:, :, 1].argmax()][0])
        c = max(contRed, key=cv2.contourArea)
        extTopRed = tuple(c[c[:, :, 1].argmin()][0])
        xbR, ybR = extBotRed
        xtR, ytR = extTopRed
    except:
        # Save error info
        errorImage.append(imgx)
        errorFile.append(fileloc)
        print('Red Area - ' + fileloc)
        continue

    # Paint Red Area wth Gray 
    cv2.fillPoly(imgf, contRed, color=(50,50,50))

    # Define cut points
    pbx = xbR
    ptx = xbR + 200
    pby = ybR + 70
    pty = ytR + 20
    dy = pby - pty

    # Define ROI
    if(((len(contRed) > 1) and (dy < 80)) or (dy < 80)):
        trROI = imgf[100:320, pbx:ptx]
        dy = 220
    else:
        trROI = imgf[pty:pby, pbx:ptx] 
        
    # Scale image 
    height, width = trROI.shape[:2]
    newH = 300
    newW = int((newH/height)*width)
    imgb = cv2.resize(trROI, (newW, newH), interpolation = cv2.INTER_CUBIC)
    
    
    # ------------------------------------------------------------- # 
    # PRE-PROCESSING - TOP HAT, HOMOMORPHIC 0.05, HOMOMORPHIC 0.5
    # ------------------------------------------------------------- #

    for case in range(0,3): 
            
         # Tophat
        if (case == 0):
            
            kernel_top = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,13))
            x = cv2.cvtColor(imgb, cv2.COLOR_BGR2GRAY)
            tophat = cv2.morphologyEx(x, cv2.MORPH_TOPHAT, kernel_top)
            result = cv2.multiply(8, tophat)
            ret, otsu = cv2.threshold(result,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # Homomorphic Filter
        if (case == 1):
            
            imgc = np.float32(imgb)
            imgc = imgc/255

            rows,cols,dim = imgc.shape
            #rh, rl, cutoff = 3,0.15,60
            rh, rl, cutoff = 2.5,0.05,38

            imgYCrCb = cv2.cvtColor(imgc, cv2.COLOR_BGR2YCrCb)
            y,cr,cb = cv2.split(imgYCrCb)

            y_log = np.log(y+0.01)
            y_fft = np.fft.fft2(y_log)
            y_fft_shift = np.fft.fftshift(y_fft)

            DX = cols/cutoff
            G = np.ones((rows,cols))
            for i in range(rows):
                for j in range(cols):
                    G[i][j]=((rh-rl)*(1-np.exp(-((i-rows/2)**2+(j-cols/2)**2)/(2*DX**2))))+rl

            result_filter = G * y_fft_shift
            result_interm = np.real(np.fft.ifft2(np.fft.ifftshift(result_filter)))
            result = np.exp(result_interm).astype('uint8')
            ret, otsu = cv2.threshold(result,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
       
        # Clahe filter
        if (case == 2):
            kernel = np.ones((3,3),np.uint8)
            aux = improved_light(imgb,3,35) #clahe
            # Otsu and Closing
            ret, otsu = cv2.threshold(aux,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            otsu = cv2.morphologyEx(otsu, cv2.MORPH_CLOSE, kernel)            
                        
        # Transform in square
        height, width = otsu.shape[:2]
        if height > width:
            newsq = np.zeros((height, width))
            square = cv2.resize(otsu, (width, width), interpolation = cv2.INTER_CUBIC)
        else:
            square = otsu[0:height, 0:height]

        # Add blank space below for reference point
        if height > width:
            if(dy > 200):
                newsq = np.pad(square, ((0,height-width),(0,0)), mode='constant')
            else:
                newsq = np.pad(square, ((0,height-width+40),(0,0)), mode='constant')
        else:
            newsq = np.pad(square, ((0,int((width-height)*0.7)),(0,0)), mode='constant')

     # ------------------------------------------------------- #              
     # REMOVE SMALL OBJECTS
     # ------------------------------------------------------- #      
    
        # Find connected points
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(newsq, connectivity=4)
        sizes = stats[1:, -1]; nb_components = nb_components - 1

        # Define minimum size
        min_size = 230

        # Creates output image (cleaned)
        h, w = output.shape
        cleaned = np.zeros([int(h), int(w), 3], dtype=np.uint8)

        # Keeps only bigger objects
        for i in range(0, nb_components):
            if sizes[i] >= min_size:
                cleaned[output == i + 1] = 255
                
                
     # ------------------------------------------------------- #                  
     # FIND THE CHARACTERS
     # ------------------------------------------------------- #      
        
        height, width = cleaned.shape[:2]
        canvas = cleaned.copy()
        cleang = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)

        # Find contours
        contours, hierarchy = cv2.findContours(cleang, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Get the actual inner list of hierarchy descriptions
        if(len(contours) == 0):
            zero += 1
            pass
        else:
            hierarchy = hierarchy[0]
            
            # Filter contours by the outermost parent components
            contours = [c[0] for c in zip(contours, hierarchy) if c[1][3] < 0]

            # Order contours 
            boundingBoxes = [cv2.boundingRect(c) for c in contours]
            (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes), key=lambda b: b[1][0], reverse=True))

        # Filter contours accordingly with size and proportion features
        caracteres = []    
        crops = []
        widths = []
        heights = []

        for currentContour in contours: 

            # Get features with EnclosingCircle and minAreaRect    
            (x,y), radius = cv2.minEnclosingCircle(currentContour)
            testBox = cv2.minAreaRect(currentContour)
            (centerbox, (wbox, hbox), angbox) = testBox 

            # Filter per size and proportion
            if (x < width/3) and (y > height*2/3):
                continue
            if (x > width*2/3) and (y < height/3):
                continue
            if radius < 14:
                continue
            if radius > 50:
                continue
            if hbox > 2.5*wbox:
                continue   
            if 2.5 < hbox/wbox < 3:
                continue

            center = (int(x),int(y))
            radius = int(radius)
            circle = (center, radius)

            # Find rectangles arround contours and aligned with the reference point 
            rad = math.atan2(height-y, 0-x)
            angle = math.degrees(rad) - 90
            bbox, crop = box_w_angle(canvas, (0, height), centerbox, radius+2, angle)

            # Resize to uniform crops
            try:    
                crop = cv2.resize(crop, (120, 180), interpolation = cv2.INTER_CUBIC) 
            except:
                print('Crops - ' + fileloc)
                pass

            # Keeps gray image
            crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

            caracteres.append(circle)
            crops.append(crop)
            widths.append(120)
            heights.append(180)
                     
     # ------------------------------------------------------- #                              
     # CREATE FINAL IMAGE
     # ------------------------------------------------------- #      
        
        # Find sum of character widths (+ space between)
        finalW = 0
        for w in widths:
            finalW = finalW + w + 5

        # Find max height of characters
        finalH = 0
        for h in heights:
            if finalH > h:
                continue
            finalH = h

        # Create an array big enough
        vis = np.zeros([int(finalH), int(finalW)], dtype=np.uint8)

        # Broadcast characters within the array
        y = 0
        for im in reversed(crops):
            try:
                h,w = im.shape
                im = cv2.resize(im, (w, finalH), interpolation = cv2.INTER_CUBIC) 
                vis[0:finalH+0, y:y+w] = im
                y += w + 5
            except:
                print('Final Broad - ' + fileloc)
                pass      

        # Invert colors
        retf, final = cv2.threshold(vis, 127, 255, cv2.THRESH_BINARY_INV)
        
     # ------------------------------------------------------- #        
     # PYTESSERACT CLASSIFIER
     # ------------------------------------------------------- #      
        
        # Pytesseract
        try:
            ocr = pytesseract.image_to_string(final, config='--psm 6 -c tessedit_char_whitelist=0123456789ABCFGT')
        except:
            print(fileloc)
     # ------------------------------------------------------- #                              
     # EVALUATE OCR AND SAVE
     # ------------------------------------------------------- #      
        
        # Get real Coil ID from file name
        splits = fileloc.rsplit('\\', 1)
        coil = splits[1].rsplit('-', 3)[1]
    
        # Find where to create the folder
        location = splits[0]

        # Create name
        name = coil + ' - ' + ocr + '.png' 
        nameOrig = coil + ' - ' + splits[1]
    
        # OCR Succeeded
        if ((coil == ocr)
        or (coil[1:] == ocr)
        or (coil[2:] == ocr[2:])
        or (len(ocr) > 6 and (coil[2:] == ocr[2:]))
        or (len(ocr) > 6 and (coil == ocr[:-1]))
        or (coil == ocr[:-1])
        or (coil[1:] == ocr[:-1])
        or (coil[2:] == ocr[:-1])
        or (len(ocr) < 7 and (coil == ocr[2:]))):
        
            # Create folder
            try:
                fileLocation = os.path.join(location, 'cascade1 Tess - Success Files')
                os.mkdir(fileLocation)
            except WindowsError:   
                pass

            # Write results
            try:
                fileName = os.path.join(fileLocation, name)     
                cv2.imwrite(fileName, final)
            except WindowsError:
                print('Write Succ - ' + fileloc)  
                pass
            
            # Case Rate Control
            if (case == 0):
                case0 += 1
            if (case == 1):
                case1 += 1
            if (case == 2):
                case2 += 1
            
            # Success Rate Counter
            count += 1
    
            # Skip other cases:
            break
        
        # OCR Failed
        else:
            
            if(case == 0 or case == 1):
                continue
            
            # Failed all pre-processing layers, save FAIL
            if(case == 2):
                
                # Create folder
                try:
                    fileLocation = os.path.join(location, 'cascade1 Tess - Fail Files')
                    os.mkdir(fileLocation)
                except WindowsError:
                    pass

                # Write results
                try:
                    fileName = os.path.join(fileLocation, name)     
                    cv2.imwrite(fileName, final)
                    fileNameOrig = os.path.join(fileLocation, nameOrig)     
                    cv2.imwrite(fileNameOrig, imgx)
                except WindowsError:
                    print('Write Fails - ' + fileloc)  
                    pass
 
    # ------------------------------------------------------- #      
    # PROGRESS PRINT
    # ------------------------------------------------------- #  
    
    progress += 1
    print("Progress: {:2.1%} - Elapsed time: {:.2f} seconds".format(progress/total, time.time() - mid_time), end="\r")

# ------------------------------------------------------- #      
# RATES PRINTS
# ------------------------------------------------------- #  
    
# PRINT TIME TO PROCESS IMAGES
total_time = time.time() - mid_time
print("Time to process images: {:.2f} seconds              ".format(total_time))
print("Time per image = {:.4f} seconds".format(total_time/progress))
print(" ")

# PRINT ACQUIRED RATES
print('Success Files = ' + str(count))
print('Total Files = ' + str(progress))
print('Accuracy = {:.2f} %'.format(count/progress*100))
print(' ')
print('Pre-Processing Used:')
print('    TopHat: {:.2f} %'.format(case0/count*100))
print('    Homo Dark: {:.2f} %'.format(case1/count*100))
print('    Homo Bright: {:.2f} %'.format(case2/count*100))
print('    Homo LowContrast: {:.2f} %'.format(case3/count*100))

D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-45-22-F909003-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-45-22-F909003-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-52-12-F909005-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-56-47-F909006-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-14-03-23-F909011-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-14-56-03-F909033-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-11-32-F909039-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-13-54-F909032-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-16-47-F909040-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-21-49-F909041-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-23-17-F909042-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-29-41-F909043-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-31

## Main Loop - TopHat + HMA + Clahe

In [25]:
# SAVE PROCESSING START TIME
mid_time = time.time()
progress = 0

# Auxiliary Counters
count = 0
case0 = 0
case1 = 0
case2 = 0
case3 = 0
zero = 0

errorImage = []
errorFile = []

# ------------------------------------------------------- #  
# MAIN LOOP
# ------------------------------------------------------- #  

for fileloc in filenames:
        
    # READ IMAGE
   
    try:
        imgx = cv2.imread(fileloc)
        imgf = imgx.copy()
    except:
        print('Read/Copy Img - ' + fileloc)
        continue

    # ------------------------------------------------------- #  
    # ROI EXTRACTION
    # ------------------------------------------------------- #  
    
    # Split Channels
    b,g,r = cv2.split(imgf)
    red = r - g 
    blue = b - g

    # Dilate/Threshold to get 1 contour only
    kernel_dil = np.ones((3,3),np.uint8)
    red = cv2.dilate(red, kernel_dil, iterations = 3)
    rets, thRed = cv2.threshold(red, 100, 255, cv2.THRESH_BINARY)

    # Find red contour
    contRed, hierqr = cv2.findContours(thRed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Order contours by size
    if(len(contRed) > 1):
        contRed = sorted(contRed, key=lambda x: cv2.contourArea(x))

    # Find extreme points of Red Area
    try:
        c = max(contRed, key=cv2.contourArea)
        extBotRed = tuple(c[c[:, :, 1].argmax()][0])
        c = max(contRed, key=cv2.contourArea)
        extTopRed = tuple(c[c[:, :, 1].argmin()][0])
        xbR, ybR = extBotRed
        xtR, ytR = extTopRed
    except:
        # Save error info
        errorImage.append(imgx)
        errorFile.append(fileloc)
        print('Red Area - ' + fileloc)
        continue

    # Paint Red Area wth Gray 
    cv2.fillPoly(imgf, contRed, color=(50,50,50))

    # Define cut points
    pbx = xbR
    ptx = xbR + 200
    pby = ybR + 70
    pty = ytR + 20
    dy = pby - pty

    # Define ROI
    if(((len(contRed) > 1) and (dy < 80)) or (dy < 80)):
        trROI = imgf[100:320, pbx:ptx]
        dy = 220
    else:
        trROI = imgf[pty:pby, pbx:ptx] 
        
    # Scale image 
    height, width = trROI.shape[:2]
    newH = 300
    newW = int((newH/height)*width)
    imgb = cv2.resize(trROI, (newW, newH), interpolation = cv2.INTER_CUBIC)
    
    
    # ------------------------------------------------------------- # 
    # PRE-PROCESSING - TOP HAT, HOMOMORPHIC 0.05, HOMOMORPHIC 0.5
    # ------------------------------------------------------------- #

    for case in range(0,3): 
            
            # Tophat
        if (case == 0):
            
            kernel_top = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,13))
            x = cv2.cvtColor(imgb, cv2.COLOR_BGR2GRAY)
            tophat = cv2.morphologyEx(x, cv2.MORPH_TOPHAT, kernel_top)
            result = cv2.multiply(8, tophat)
            ret, otsu = cv2.threshold(result,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # HMA
        if (case == 1):
            
            imgc = np.float32(imgb)
            imgc = imgc/255

            rows,cols,dim = imgc.shape
            rh, rl, cutoff = 2.5,0.05,38

            imgYCrCb = cv2.cvtColor(imgc, cv2.COLOR_BGR2YCrCb)
            y,cr,cb = cv2.split(imgYCrCb)

            y_log = np.log(y+0.01)
            y_fft = np.fft.fft2(y_log)
            y_fft_shift = np.fft.fftshift(y_fft)

            DX = cols/cutoff
            G = np.ones((rows,cols))
            for i in range(rows):
                for j in range(cols):
                    G[i][j]=((rh-rl)*(1-np.exp(-((i-rows/2)**2+(j-cols/2)**2)/(2*DX**2))))+rl

            result_filter = G * y_fft_shift
            result_interm = np.real(np.fft.ifft2(np.fft.ifftshift(result_filter)))
            result = np.exp(result_interm).astype('uint8')
            ret, otsu = cv2.threshold(result,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
       
        # HMB
        if (case == 2):
            
            imgc = np.float32(imgb)
            imgc = imgc/255

            rows,cols,dim = imgc.shape

            rh, rl, cutoff = 2.8,0.5,60

            imgYCrCb = cv2.cvtColor(imgc, cv2.COLOR_BGR2YCrCb)
            y,cr,cb = cv2.split(imgYCrCb)

            y_log = np.log(y+0.01)
            y_fft = np.fft.fft2(y_log)
            y_fft_shift = np.fft.fftshift(y_fft)

            DX = cols/cutoff
            G = np.ones((rows,cols))
            for i in range(rows):
                for j in range(cols):
                    G[i][j]=((rh-rl)*(1-np.exp(-((i-rows/2)**2+(j-cols/2)**2)/(2*DX**2))))+rl

            result_filter = G * y_fft_shift
            result_interm = np.real(np.fft.ifft2(np.fft.ifftshift(result_filter)))
            result = np.exp(result_interm).astype('uint8')
            ret, otsu = cv2.threshold(result,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # Transform in square
        height, width = otsu.shape[:2]
        if height > width:
            newsq = np.zeros((height, width))
            square = cv2.resize(otsu, (width, width), interpolation = cv2.INTER_CUBIC)
        else:
            square = otsu[0:height, 0:height]

        # Add blank space below for reference point
        if height > width:
            if(dy > 200):
                newsq = np.pad(square, ((0,height-width),(0,0)), mode='constant')
            else:
                newsq = np.pad(square, ((0,height-width+40),(0,0)), mode='constant')
        else:
            newsq = np.pad(square, ((0,int((width-height)*0.7)),(0,0)), mode='constant')

     # ------------------------------------------------------- #              
     # REMOVE SMALL OBJECTS
     # ------------------------------------------------------- #      
    
        # Find connected points
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(newsq, connectivity=4)
        sizes = stats[1:, -1]; nb_components = nb_components - 1

        # Define minimum size
        min_size = 230

        # Creates output image (cleaned)
        h, w = output.shape
        cleaned = np.zeros([int(h), int(w), 3], dtype=np.uint8)

        # Keeps only bigger objects
        for i in range(0, nb_components):
            if sizes[i] >= min_size:
                cleaned[output == i + 1] = 255
                
                
     # ------------------------------------------------------- #                  
     # FIND THE CHARACTERS
     # ------------------------------------------------------- #      
        
        height, width = cleaned.shape[:2]
        canvas = cleaned.copy()
        cleang = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)

        # Find contours
        contours, hierarchy = cv2.findContours(cleang, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Get the actual inner list of hierarchy descriptions
        if(len(contours) == 0):
            zero += 1
            pass
        else:
            hierarchy = hierarchy[0]
            
            # Filter contours by the outermost parent components
            contours = [c[0] for c in zip(contours, hierarchy) if c[1][3] < 0]

            # Order contours 
            boundingBoxes = [cv2.boundingRect(c) for c in contours]
            (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes), key=lambda b: b[1][0], reverse=True))

        # Filter contours accordingly with size and proportion features
        caracteres = []    
        crops = []
        widths = []
        heights = []

        for currentContour in contours: 

            # Get features with EnclosingCircle and minAreaRect    
            (x,y), radius = cv2.minEnclosingCircle(currentContour)
            testBox = cv2.minAreaRect(currentContour)
            (centerbox, (wbox, hbox), angbox) = testBox 

            # Filter per size and proportion
            if (x < width/3) and (y > height*2/3):
                continue
            if (x > width*2/3) and (y < height/3):
                continue
            if radius < 14:
                continue
            if radius > 50:
                continue
            if hbox > 2.5*wbox:
                continue   
            if 2.5 < hbox/wbox < 3:
                continue

            center = (int(x),int(y))
            radius = int(radius)
            circle = (center, radius)

            # Find rectangles arround contours and aligned with the reference point 
            rad = math.atan2(height-y, 0-x)
            angle = math.degrees(rad) - 90
            bbox, crop = box_w_angle(canvas, (0, height), centerbox, radius+2, angle)

            # Resize to uniform crops
            try:    
                crop = cv2.resize(crop, (120, 180), interpolation = cv2.INTER_CUBIC) 
            except:
                print('Crops - ' + fileloc)
                pass

            # Keeps gray image
            crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

            caracteres.append(circle)
            crops.append(crop)
            widths.append(120)
            heights.append(180)
                     
     # ------------------------------------------------------- #                              
     # CREATE FINAL IMAGE
     # ------------------------------------------------------- #      
        
        # Find sum of character widths (+ space between)
        finalW = 0
        for w in widths:
            finalW = finalW + w + 5

        # Find max height of characters
        finalH = 0
        for h in heights:
            if finalH > h:
                continue
            finalH = h

        # Create an array big enough
        vis = np.zeros([int(finalH), int(finalW)], dtype=np.uint8)

        # Broadcast characters within the array
        y = 0
        for im in reversed(crops):
            try:
                h,w = im.shape
                im = cv2.resize(im, (w, finalH), interpolation = cv2.INTER_CUBIC) 
                vis[0:finalH+0, y:y+w] = im
                y += w + 5
            except:
                print('Final Broad - ' + fileloc)
                pass      

        # Invert colors
        retf, final = cv2.threshold(vis, 127, 255, cv2.THRESH_BINARY_INV)
        
     # ------------------------------------------------------- #        
     # PYTESSERACT CLASSIFIER
     # ------------------------------------------------------- #      
        
        # Pytesseract
        try:
            ocr = pytesseract.image_to_string(final, config='--psm 6 -c tessedit_char_whitelist=0123456789ABCFGT')
        except:
            print(fileloc)
     # ------------------------------------------------------- #                              
     # EVALUATE OCR AND SAVE
     # ------------------------------------------------------- #      
        
        # Get real Coil ID from file name
        splits = fileloc.rsplit('\\', 1)
        coil = splits[1].rsplit('-', 3)[1]
    
        # Find where to create the folder
        location = splits[0]

        # Create name
        name = coil + ' - ' + ocr + '.png' 
        nameOrig = coil + ' - ' + splits[1]
    
        # OCR Succeeded
        if ((coil == ocr)
        or (coil[1:] == ocr)
        or (coil[2:] == ocr[2:])
        or (len(ocr) > 6 and (coil[2:] == ocr[2:]))
        or (len(ocr) > 6 and (coil == ocr[:-1]))
        or (coil == ocr[:-1])
        or (coil[1:] == ocr[:-1])
        or (coil[2:] == ocr[:-1])
        or (len(ocr) < 7 and (coil == ocr[2:]))):
        
            # Create folder
            try:
                fileLocation = os.path.join(location, 'cascade2 Tess - Success Files')
                os.mkdir(fileLocation)
            except WindowsError:   
                pass

            # Write results
            try:
                fileName = os.path.join(fileLocation, name)     
                cv2.imwrite(fileName, final)
            except WindowsError:
                print('Write Succ - ' + fileloc)  
                pass
            
            # Case Rate Control
            if (case == 0):
                case0 += 1
            if (case == 1):
                case1 += 1
            if (case == 2):
                case2 += 1
            
            # Success Rate Counter
            count += 1
    
            # Skip other cases:
            break
        
        # OCR Failed
        else:
            
            if(case == 0 or case == 1):
                continue
            
            # Failed all pre-processing layers, save FAIL
            if(case == 2):
                
                # Create folder
                try:
                    fileLocation = os.path.join(location, 'cascade2 Tess - Fail Files')
                    os.mkdir(fileLocation)
                except WindowsError:
                    pass

                # Write results
                try:
                    fileName = os.path.join(fileLocation, name)     
                    cv2.imwrite(fileName, final)
                    fileNameOrig = os.path.join(fileLocation, nameOrig)     
                    cv2.imwrite(fileNameOrig, imgx)
                except WindowsError:
                    print('Write Fails - ' + fileloc)  
                    pass
 
    # ------------------------------------------------------- #      
    # PROGRESS PRINT
    # ------------------------------------------------------- #  
    
    progress += 1
    print("Progress: {:2.1%} - Elapsed time: {:.2f} seconds".format(progress/total, time.time() - mid_time), end="\r")

# ------------------------------------------------------- #      
# RATES PRINTS
# ------------------------------------------------------- #  
    
# PRINT TIME TO PROCESS IMAGES
total_time = time.time() - mid_time
print("Time to process images: {:.2f} seconds              ".format(total_time))
print("Time per image = {:.4f} seconds".format(total_time/progress))
print(" ")

# PRINT ACQUIRED RATES
print('Success Files = ' + str(count))
print('Total Files = ' + str(progress))
print('Accuracy = {:.2f} %'.format(count/progress*100))
print(' ')
print('Pre-Processing Used:')
print('    TopHat: {:.2f} %'.format(case0/count*100))
print('    Homo Dark: {:.2f} %'.format(case1/count*100))
print('    Homo Bright: {:.2f} %'.format(case2/count*100))
print('    Homo LowContrast: {:.2f} %'.format(case3/count*100))

D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-45-22-F909003-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-53-45-F909007-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-13-56-47-F909006-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-14-03-23-F909011-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-14-56-03-F909033-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-11-32-F909039-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-13-54-F909032-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-16-47-F909040-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-15-23-17-F909042-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-16-10-44-F909060-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-16-14-25-F909052-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-17-33-09-F909103-3-markedup.png
D:\UFES\RevisionWorkstation\Benchmark\17-09-21-18-10